In [1]:
# load module
import numpy as np
import numpy.matlib
import math
import scipy.io as sio
import warnings
from scipy import signal
warnings.filterwarnings('default')

In [2]:
def corr2(a, b):
    """ Solving for the two-dimensional correlation coefficient
    :param a: 
    :param b: 

    :return: correlation coefficient
    """
    
    a = a - np.sum(a) / np.size(a)
    b = b - np.sum(b) / np.size(b)
    r = (a * b).sum() / math.sqrt((a * a).sum() * (b * b).sum())
    return r

def acc_calculate(predict):
    """ Calculate accuracy
    :param predict:  (n_trial,n_event)

    :return: acc
    """
    
    [nTrials, nEvents] = predict.shape  
    label_target = np.ones((nTrials, 1)) * np.arange(0, nEvents, 1, int)
    logical_right = (label_target == predict)
    acc_num = np.sum(logical_right != 0)
    acc = acc_num / nTrials / nEvents
    return acc

class PreProcessing_BETA():
    """
    Adapted from Orion Han
    https://github.com/OrionHH/BrainOn-an-online-brain-computer-interface-BCI-framework
    """
    
    CHANNELS = [
        'FP1','FPZ','FP2','AF3','AF4','F7','F5','F3',
        'F1','FZ','F2','F4','F6','F8','FT7','FC5',
        'FC3','FC1','FCZ','FC2','FC4','FC6','FC8','T7',
        'C5','C3','C1','CZ','C2','C4','C6','T8',
        'M1','TP7','CP5','CP3','CP1','CPZ','CP2','CP4',
        'CP6','TP8','M2','P7','P5','P3','P1','PZ',
        'P2','P4','P6','P8','PO7','PO5','PO3','POZ',
        'PO4','PO6','PO8','CB1','O1','OZ','O2','CB2'
    ] # M1: 33. M2: 43.

    def __init__(self, filepath,  t_begin, t_end, n_classes=40, fs_down=250, chans=None, num_filter=1):

        self.filepath = filepath
        self.fs_down = fs_down
        self.t_begin = t_begin
        self.t_end = t_end
        self.chans = chans
        self.n_classes = n_classes
        self.num_filter = num_filter

    def load_data(self):
        '''
        Application: load data and selected channels by chans.

        :param chans: list | None
        :return: raw_data: 4-D, numpy
            n_chans * n_samples * n_classes * n_trials

        '''
        raw_mat = sio.loadmat(self.filepath)
        raw_data11 = raw_mat['data']  
        data = raw_data11[0,0]['EEG']
        raw_data = np.transpose(data,[0,1,3,2]) # n_chans * n_samples * n_classes * n_trials

        idx_loc = list()
        if isinstance(self.chans, list):
            for _, char_value in enumerate(self.chans):
                idx_loc.append(self.CHANNELS.index(char_value.upper()))

        raw_data = raw_data[idx_loc, : , : , :] if idx_loc else raw_data

        self.raw_fs = 250  # .mat sampling rate

        return raw_data

    def resample_data(self, raw_data):
        '''
        :param raw_data: from method load_data.
        :return: raw_data_resampled, 4-D, numpy
            n_chans * n_samples * n_classes * n_trials
        '''
        if self.raw_fs > self.fs_down:
            raw_data_resampled = signal.resample(raw_data, round(self.fs_down*raw_data.shape[1]/self.raw_fs), axis=1)
        elif self.raw_fs < self.fs_down:
            warnings.warn('You are up-sampling, no recommended')
            raw_data_resampled = signal.resample(raw_data, round(self.fs_down*raw_data.shape[1]/self.raw_fs), axis=1)
        else:
            raw_data_resampled = raw_data

        return raw_data_resampled

    def _get_iir_sos_band(self, w_pass, w_stop):
        '''
        Get second-order sections (like 'ba') of Chebyshev type I filter.
        :param w_pass: list, 2 elements
        :param w_stop: list, 2 elements
        :return: sos_system
            i.e the filter coefficients.
        '''
        if len(w_pass) != 2 or len(w_stop) != 2:
            raise ValueError('w_pass and w_stop must be a list with 2 elements.')

        if w_pass[0] > w_pass[1] or w_stop[0] > w_stop[1]:
            raise ValueError('Element 1 must be greater than Element 0 for w_pass and w_stop.')

        if w_pass[0] < w_stop[0] or w_pass[1] > w_stop[1]:
            raise ValueError('It\'s a band-pass iir filter, please check the values between w_pass and w_stop.')

        wp = [2 * w_pass[0] / self.fs_down, 2 * w_pass[1] / self.fs_down]
        ws = [2 * w_stop[0] / self.fs_down, 2 * w_stop[1] / self.fs_down]
        gpass = 4  
        gstop = 30  # dB

        N, wn = signal.cheb1ord(wp, ws, gpass=gpass, gstop=gstop)
        sos_system = signal.cheby1(N, rp=0.5, Wn=wn, btype='bandpass', output='sos')

        return sos_system


    def filtered_data_iir111(self, w_pass_2d, w_stop_2d, data):
        '''
        filter data by IIR, which parameters are set by method _get_iir_sos_band in BasePreProcessing class.
        :param w_pass_2d: 2-d, numpy,
            w_pass_2d[0, :]: w_pass[0] of method _get_iir_sos_band,
            w_pass_2d[1, :]: w_pass[1] of method _get_iir_sos_band.
        :param w_stop_2d: 2-d, numpy,
            w_stop_2d[0, :]: w_stop[0] of method _get_iir_sos_band,
            w_stop_2d[1, :]: w_stop[1] of method _get_iir_sos_band.
        :param data: 4-d, numpy, from method load_data or resample_data.
            n_chans * n_samples * n_classes * n_trials
        :return: filtered_data: dict,
            {'bank1': values1, 'bank2': values2, ...,'bank'+str(num_filter): values}
            values1, values2,...: 4-D, numpy, n_chans * n_samples * n_classes * n_trials.
        e.g.
        w_pass_2d = np.array([[5, 14, 22, 30, 38, 46, 54],[70, 70, 70, 70, 70, 70, 70]])
        w_stop_2d = np.array([[3, 12, 20, 28, 36, 44, 52],[72, 72, 72, 72, 72, 72, 72]])
        '''
        if w_pass_2d.shape != w_stop_2d.shape:
            raise ValueError('The shape of w_pass_2d and w_stop_2d should be equal.')
        if self.num_filter > w_pass_2d.shape[1]:
            raise ValueError('num_filter should be less than or equal to w_pass_2d.shape[1]')

        begin_point, end_point = int(np.ceil(self.t_begin * self.fs_down)), int(np.ceil(self.t_end * self.fs_down) + 1)
        data = data[:,begin_point:end_point,:,:]

        sos_system = dict()
        filtered_data = dict()
        for idx_filter in range(self.num_filter):
            sos_system['filter'+str(idx_filter+1)] = self._get_iir_sos_band(w_pass=[w_pass_2d[0, idx_filter], w_pass_2d[1, idx_filter]],
                                                                            w_stop=[w_stop_2d[0, idx_filter],
                                                                                    w_stop_2d[1, idx_filter]])
            filter_data = signal.sosfiltfilt(sos_system['filter' + str(idx_filter + 1)], data, axis=1)
            filtered_data['bank'+str(idx_filter+1)] = filter_data

        return filtered_data

/root/miniconda3/envs/myconda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### SAME

In [3]:
def TRCs_estimation(data, mean_target):
    """ source signal estimation

    :param data: n_channel_1, n_times
    :param mean_target: n_channel_2, n_times

    :return: data_after: n_channel_2, n_times
    """
    
    nChannel, nTimes = np.shape(data)
    X_a = data
    X = mean_target

    # solve PT
    PT = X @ X_a.T @ np.linalg.inv(X_a @ X_a.T)
    data_after = PT @ X_a

    return data_after

def get_augment_fb_noiseAfter(fs, f, Nh_start, Nh_end, ntrail_noise, mean_temp):
    """  Artificially generated signals by SAME

    :param fs: Sampling rate
    :param f:  the frequency of signal
    :param Nh_start: Minimum number of harmonics
    :param Nh_end: Maximum number of harmonics
    :param ntrial_noise: Number of generated signals
    :param mean_temp: n_channel, n_times

    :return: data_aug: n_channel, n_times, ntrial_noise
    """
    
    Nh_step  = np.arange(Nh_start,Nh_end+1,1)
    Nh = Nh_step.shape[-1]
    nChannel, nTime = mean_temp.shape
    #  Generate reference signal Yf
    Ts = 1 / fs
    n = np.arange(nTime) * Ts
    Yf = np.zeros((nTime, 2*Nh))   
    flag = 0
    for iNh in Nh_step:
        y_sin = np.sin(2 * np.pi * f * iNh * n)
        Yf[:, flag * 2] = y_sin
        y_cos = np.cos(2 * np.pi * f * iNh  * n)
        Yf[:, flag * 2 + 1] = y_cos
        flag = flag + 1

    Z = TRCs_estimation(Yf.T, mean_temp)
    # get vars of Z
    vars = np.zeros((Z.shape[0],Z.shape[0]))
    for i_c in range(nChannel):
        vars[i_c,i_c] = np.var(Z[i_c,:])

    # add noise
    data_aug = np.zeros((nChannel, nTime, ntrail_noise))
    for i_aug in range(ntrail_noise):
        # Randomly generated noise
        Datanosie = np.random.multivariate_normal(mean=np.zeros((nChannel)), cov=vars, size =  nTime)
        data_aug[:,:,i_aug] = Z + 0.05 * Datanosie.T

    return data_aug

### eTRCA

In [4]:
##TRCA
def trca_matrix(data):
    """ Task-related component analysis (TRCA)
    :param data: Multi-trial EEG signals under the same task
           ndarray(n_channels, n_sample_points, n_trials)

    :return: w: spatial filter
           ndarray(n_channels, 1)
    """
    
    X = data

    # X_mean = X.mean(axis=1, keepdims=True)
    # X = X - X_mean

    nChans = X.shape[0]
    nTimes = X.shape[1]
    nTrial = X.shape[2]
    #  solve S
    S = np.zeros((nChans, nChans))
    for i in range(nTrial):
        for j in range(nTrial):
            if (i != j):
                x_i = X[:, :, i]
                x_j = X[:, :, j]
                S = S + np.dot(x_i, (x_j.T))
    #  solve Q
    X1 = X.reshape([nChans, nTimes * nTrial], order='F')  
    Q = X1 @ X1.T
    
    #  get eigenvector
    b = np.dot(np.linalg.inv(Q), S)
    [eig_value, eig_w] = np.linalg.eig(b)  # in matlab：a/b = inv(a)*b

    # Descending order
    eig_w = eig_w[:, eig_value.argsort()[::-1]]  # return indices in ascending order and reverse
    eig_value.sort()
    eig_value = eig_value[::-1]  # sort in descending

    w = eig_w[:, 0]
    return w.real

def TRCA_train(trainData):
    """ Get TRCA spatial filters and average templates for all classes
    :param trainData: training data of all events
            ndarray(n_channels, n_sample_points, n_events, n_trials)

    :return: w: (n_channels, n_events)
             mean_temp (n_channels, n_sample_points, n_events)
    """
    
    [nChannels, nTimes, nEvents, nTrials] = trainData.shape 
    # get w of event class
    w = np.zeros((nChannels, nEvents))
    for i in range(nEvents):
        w_data = trainData[:, :, i, :]
        w1 = trca_matrix(w_data)
        w[:, i] = w1
    # get mean temps
    mean_temp = np.zeros((nChannels, nTimes, nEvents))
    mean_temp = np.mean(trainData, -1)
    return w, mean_temp

def TRCA_test(testData, w, mean_temp, ensemble):
    """
    :param testData: test_data of multi trials
           ndarray(n_channels, n_sample_points, n_trials(equals to n_events))
    :param w: Spatial Filters
           ndarray (n_channels, n_events)
    :param mean_temp: Average template
           ndarray (n_channels, n_sample_points, n_events)
    :param ensemble: bool

    :return: predict of singe block
           ndarray(n_trials, n_classes)
    """
    
    [nChannels, nTimes, nEvents] = testData.shape
    rr = np.zeros((nEvents, nEvents))
    for m in range(nEvents):  # the m-th test data
        test = testData[:, :, m]
        # Calculate the vector of correlation coefficients
        r = np.zeros(nEvents)
        for n in range(nEvents):  # the n-th train model
            train = mean_temp[:, :, n]
            if ensemble is True:
                r[n] = corr2(train.T @ w, test.T @ w)
            else:
                r[n] = corr2(train.T @ w[:, n], test.T @ w[:, n])
        rr[m, :] = r
    return rr

### TDCA

In [5]:
##TDCA
def get_P(f_list, Nh, sTime, sfreq):
    """ Get the projection matrix P for all classes
    :param f_list: the frequency of all events
    :param Nh: number of harmonics
    :param sTime: signal duration
    :param sfreq: sampling rate

    :return: P: the projection matrix P for all classes
             ndarray(n_Times, n_Times, n_Events)
    """
    
    nEvent = f_list.shape[0]
    P = np.zeros((int(sTime * sfreq), int(sTime * sfreq), nEvent))
    for iievent in range(nEvent):
        #  Generate reference signal Yf
        f = f_list[iievent]
        nTime = int(sTime * sfreq)
        Ts = 1 / sfreq
        n = np.arange(nTime) * Ts
        Yf = np.zeros((nTime, 2 * Nh)) 
        for iNh in range(Nh):
            y_sin = np.sin(2 * np.pi * f * (iNh + 1) * n)
            Yf[:, iNh * 2] = y_sin
            y_cos = np.cos(2 * np.pi * f * (iNh + 1) * n)
            Yf[:, iNh * 2 + 1] = y_cos
        q, _ = np.linalg.qr(Yf, mode='reduced')
        P[:,:,iievent] = q @ q.T

    return P

def tdca_matrix(data, Nk):
    """ Task-discriminant component analysis (TDCA)
    :param data: training data of all events
           ndarray(n_channels * (l + 1), 2*n_points,  n_events, n_trials)
    :param Nk: the number of subspaces
           int

    :return: w: Spatial Filters
           ndarray(n_channels * (l + 1), Nk)
    """
    
    X_aug_2 = np.transpose(data,[2,3,0,1])  # nEvents, nTrials, nChannels * (l + 1), 2*npoints
    [n_events, n_trials, _, _] = X_aug_2.shape
    # get Sb
    class_center = X_aug_2.mean(axis=1)  # # nEvents , nChannels * (l + 1), 2*npoints
    total_center = class_center.mean(axis=0,keepdims=True)  # # 1, nChannels * (l + 1), 2*npoints
    Hb = class_center - total_center  # Broadcasting in numpy
    Sb = np.einsum('ecp, ehp->ch', Hb, Hb)
    Sb /= n_events
    # get Sw
    class_center = np.expand_dims(class_center, 1) # nEvents , 1, nChannels * (l + 1), 2*npoints
    Hw = X_aug_2 - np.tile(class_center,[1, n_trials,1,1]) # nEvents , nTrials, nChannels * (l + 1), 2*npoints
    Sw = np.einsum('etcp, ethp->ch', Hw, Hw)
    Sw /= (n_events * n_trials)
    Sw = 0.001 * np.eye(Hw.shape[2]) + Sw   # regularization
    #  get eigenvector
    b = np.dot(np.linalg.inv(Sw), Sb)
    [eig_value, eig_w] = np.linalg.eig(b) 

    # Descending order
    eig_w = eig_w[:, eig_value.argsort()[::-1]]  # return indices in ascending order and reverse
    eig_value.sort()
    eig_value = eig_value[::-1]  # sort in descending
    w = eig_w[:, :Nk]

    return w  

def TDCA_train(trainData, P ,l , Nk ):
    """ Get TDCA spatial filters and average templates for all classes
    :param trainData: training data of all events
           ndarray(n_channels, (n_sample_points + l), n_events, n_trials)
    :param P: projection matrix for all classes
           ndarray(n_sample_points, n_sample_points, n_events)
    :param l: delay point
    :param Nk: the number of subspaces

    :return: w: Spatial Filters
           ndarray(n_channels * (l + 1), Nk)
             mean_temp: average templates
           ndarray(Nk, 2n_sample_points, n_events)

    """
    
    [nChannels, nTimes, nEvents, nTrials] = trainData.shape 
    npoints = nTimes - l


    data_aug_2 = np.zeros((nChannels * (l + 1), 2*npoints, nEvents, nTrials))
    for ievent in range(nEvents):
        dat = trainData[:, :, ievent,:]
        # first
        dat_aug_1 = np.zeros((nChannels*(l+1), npoints, nTrials))
        for il in range(l+1):
            dat_aug_1[il*(nChannels):(il+1)*nChannels, : ,: ] = dat[:,il:(il+npoints) ,:]
        # second
        dat_p = np.zeros_like((dat_aug_1))
        for itrial in range(nTrials):
            dat_p[:, :, itrial] = dat_aug_1[:, :, itrial] @ P[:, :, ievent]  # projection
        dat_aug_2 = np.concatenate((dat_aug_1, dat_p), axis=1, out=None)
        #
        data_aug_2[..., ievent, :] = dat_aug_2

    # get w
    w = tdca_matrix(data_aug_2 , Nk=Nk)
    # get mean temps   Nk * 2 Num of sample points * num of events
    mean_tem = np.zeros((Nk, npoints*2, nEvents))
    mean_data = np.mean(data_aug_2, -1)
    for i in range((nEvents)):
        mean_tem[:,:,i] = w.T @ mean_data[:,:,i]

    return w, mean_tem

def TDCA_test(testData, w, mean_temp ,P , l ):
    """
    :param testData: test_data of multi trials
           ndarray(n_channels, n_sample_points, n_trials(equals to n_events))
    :param w: Spatial Filters
           ndarray(n_channels * (l + 1), Nk)
    :param mean_temp: Average template
           ndarray(Nk, 2n_sample_points, n_events)
    :param P:  projection matrix for all classes
           ndarray(n_sample_points, n_sample_points, n_events)
    :param l:  delay point

    :return: predict of singe block
           ndarray(n_trials, n_classes)
    """
    
    [nChannels, nTimes, nEvents] = testData.shape
    rr = np.zeros((nEvents, nEvents))
    for m in range(nEvents):  # the m-th test data
        test = testData[:, :, m]
        # first
        test_aug_1 = np.zeros((nChannels * (l + 1), nTimes))
        aug_zero = np.zeros((nChannels, l))  # Splice 0 matrix
        test = np.concatenate((test, aug_zero), axis=1, out=None)  # nChannels, nTimes + l
        for il in range(l + 1):
            test_aug_1[il * (nChannels):(il + 1) * nChannels, :] = test[:, il:(il + nTimes)]
        # Calculate the vector of correlation coefficients
        r = np.zeros(nEvents)
        for n in range(nEvents):  # the n-th train model
            # second
            dat_p = test_aug_1 @ P[:, :, n]
            test_aug_2 = np.concatenate((test_aug_1, dat_p), axis=1, out=None)
            # slove rr
            train = mean_temp[:, :, n]
            r[n] = corr2(train, w.T @ test_aug_2)
        rr[m, :] = r
    return rr

### COUNT ACC

In [6]:
import os, sys
import scipy.io as sio
import warnings
warnings.filterwarnings('ignore')
import time
from joblib import Parallel, delayed

import numpy as np
from sklearn.model_selection import ShuffleSplit,LeaveOneOut


def beta_TDCA_Aug(idx_num, n_train, t_task ,n_Aug):

    # setting
    f_list = [8.6, 8.8,
              9, 9.2, 9.4, 9.6, 9.8,
              10, 10.2, 10.4, 10.6, 10.8,
              11, 11.2, 11.4, 11.6, 11.8,
              12, 12.2, 12.4, 12.6, 12.8,
              13, 13.2, 13.4, 13.6, 13.8,
              14, 14.2, 14.4, 14.6, 14.8,
              15, 15.2, 15.4, 15.6, 15.8,
              8, 8.2, 8.4, ]
    f_list = np.array(f_list)
    subject_id = ['S'+'{:02d}'.format(idx_subject+1) for idx_subject in range(70)] # S01,S02,.....S70

    idx_num = idx_num
    idx_subject = subject_id[idx_num]
    sfreq = 250
    filepath = r'Beta'  
    filepath = os.path.join(filepath, str(idx_subject) + '.mat')
    num_filter = 5
    preEEG = PreProcessing_BETA(filepath, t_begin=0.5, t_end=0.5 + 0.13 + t_task + 3/sfreq,
                           fs_down=250, chans=['POZ', 'PZ', 'PO3', 'PO5', 'PO4', 'PO6', 'O1', 'OZ', 'O2'],
                           num_filter=num_filter)

    raw_data = preEEG.load_data()
    w_pass_2d = np.array([[5, 14, 22, 30, 38], [90, 90, 90, 90, 90]])  # 70
    w_stop_2d = np.array([[3, 12, 20, 28, 36], [92, 92, 92, 92, 92]])  # 72
    filtered_data = preEEG.filtered_data_iir111(w_pass_2d, w_stop_2d, raw_data)

    """
     Cross-validation parameter setting
    """
    nBlock = 4
    nEvent = 40
    train_size = n_train  
    n_splits = 4
    if train_size == nBlock - 1:
        kf = LeaveOneOut()
    else:
        kf = ShuffleSplit(n_splits=n_splits, train_size=train_size, random_state=idx_num + 1)

    """
    TDCA parameter setting
    """
    l = 3  # delay point 
    t = t_task
    train_point  = np.arange(int((0.13) * sfreq), int((0.13 + t) * sfreq)+l)
    test_point = np.arange(int((0.13) * sfreq), int((0.13 + t) * sfreq))
    # get P of all classes
    P = get_P(f_list=f_list, Nh=5, sTime=t, sfreq=sfreq)

    # Cross-validation
    acc_s = 0
    for train, test in kf.split(np.arange(nBlock)):

        # train : get ensembleW of banks
        train_w = dict()
        train_meantemp = dict()
        for idx_filter in range(num_filter):
            idx_filter += 1
            bank_data = filtered_data['bank' + str(idx_filter)]
            train_data11 = bank_data[:, :, :, train]  # 
            train_data = train_data11[:, train_point, :, :]  # n_channel * n_times * n_events * n_trials
            
            if n_Aug == 0:
                trainData_pt = train_data.copy()
            else:            
                # Data augmentation
                ntrail_noise = n_Aug
                data_augment = np.zeros((train_data.shape[0], train_data.shape[1], train_data.shape[2], ntrail_noise))
                for ievent in range(nEvent):
                    # get Nh_strat
                    f = f_list[ievent]
                    for ih in range(5):
                        ih = ih+1
                        if ih*f >= 8*idx_filter:
                            Nh_start = ih
                            break
                    data_augment[:, :, ievent, :] = get_augment_fb_noiseAfter(fs=sfreq, f=f_list[ievent],Nh_start=Nh_start, Nh_end=5,
                                                                ntrail_noise=ntrail_noise,
                                                                mean_temp=np.mean(train_data,-1)[:, :, ievent])
                trainData_pt = np.concatenate((train_data, data_augment), axis=3)

            # train
            w, mean_temp_TDCA = TDCA_train(trainData_pt, P=P, l=l, Nk=9)
            #
            train_w['bank' + str(idx_filter)] = w
            train_meantemp['bank' + str(idx_filter)] = mean_temp_TDCA

        # test:
        predictAll = np.zeros((test.shape[0], nEvent))
        flag = 0
        for isplit in test:
            rrall = np.zeros((nEvent, nEvent))
            for idx_filter in range(num_filter):
                idx_filter += 1
                bank_data = filtered_data['bank' + str(idx_filter)]
                test_data111 = bank_data[:, :, :, isplit]
                test_data = test_data111[:,test_point,:]
                rr = TDCA_test(test_data, train_w['bank' + str(idx_filter)], train_meantemp['bank' + str(idx_filter)],
                               P=P, l=l)
                rrall += np.multiply(np.sign(rr), (rr ** 2)) * (idx_filter ** (-1.25) + 0.25)
            predict = np.argmax(rrall, -1)
            predictAll[flag, :] = predict
            flag += 1
        acc_s = acc_calculate(predictAll) + acc_s
    acc = acc_s / n_splits
    # print('sub', idx_num + 1, ', acc = ', acc_s / n_splits)
    return acc

def beta_eTRCA_Aug(idx_num, n_train, t_task ,n_Aug):

    # setting
    f_list = [8.6, 8.8,
              9, 9.2, 9.4, 9.6, 9.8,
              10, 10.2, 10.4, 10.6, 10.8,
              11, 11.2, 11.4, 11.6, 11.8,
              12, 12.2, 12.4, 12.6, 12.8,
              13, 13.2, 13.4, 13.6, 13.8,
              14, 14.2, 14.4, 14.6, 14.8,
              15, 15.2, 15.4, 15.6, 15.8,
              8, 8.2, 8.4, ]
    subject_id = ['S'+'{:02d}'.format(idx_subject+1) for idx_subject in range(70)]  # S01,S02,...S70

    idx_num = idx_num
    idx_subject = subject_id[idx_num]
    sfreq = 250
    filepath = r'Beta'  
    filepath = os.path.join(filepath, str(idx_subject) + '.mat')
    num_filter = 5
    preEEG = PreProcessing_BETA(filepath, t_begin=0.5, t_end=0.5 + 0.13 + t_task,  # t_begin=0.5+0.13, t_end=0.5+0.13+0.3
                           fs_down=250, chans=['POZ', 'PZ', 'PO3', 'PO5', 'PO4', 'PO6', 'O1', 'OZ', 'O2'],
                           num_filter=num_filter)

    raw_data = preEEG.load_data()
    w_pass_2d = np.array([[5, 14, 22, 30, 38], [90, 90, 90, 90, 90]])  # 70
    w_stop_2d = np.array([[3, 12, 20, 28, 36], [92, 92, 92, 92, 92]])  # 72
    filtered_data = preEEG.filtered_data_iir111(w_pass_2d, w_stop_2d, raw_data)

    """
     Cross-validation parameter setting
    """
    nBlock = 4
    nEvent = 40
    train_size = n_train 
    n_splits = 4
    if train_size == nBlock - 1:
        kf = LeaveOneOut()
    else:
        kf = ShuffleSplit(n_splits=n_splits, train_size=train_size, random_state=idx_num + 1)

    t = t_task
    task_point = np.arange(int((0.13) * sfreq), int((0.13 + t) * sfreq))

    # train : get ensembleW of banks
    acc_s = 0
    for train, test in kf.split(np.arange(nBlock)):

        # train : get ensembleW of banks
        train_w = dict()
        train_meantemp = dict()
        for idx_filter in range(num_filter):
            idx_filter += 1
            bank_data = filtered_data['bank' + str(idx_filter)]
            train_data11 = bank_data[:, :, :, train] 
            train_data = train_data11[:, task_point, :, :]  # n_channel * n_times * n_events * n_trials
            
            if n_Aug == 0:
                trainData_pt = train_data.copy()
            else:            
                # Data augmentation
                ntrail_noise = n_Aug
                data_augment = np.zeros((train_data.shape[0], train_data.shape[1], train_data.shape[2], ntrail_noise))
                for ievent in range(nEvent):
                    # get Nh_strat
                    f = f_list[ievent]
                    for ih in range(5):
                        ih = ih+1
                        if ih*f >= 8*idx_filter:
                            Nh_start = ih
                            break
                    data_augment[:, :, ievent, :] = get_augment_fb_noiseAfter(fs=sfreq, f=f_list[ievent],Nh_start=Nh_start, Nh_end=5,
                                                                ntrail_noise=ntrail_noise,
                                                                mean_temp=np.mean(train_data,-1)[:, :, ievent])
                trainData_pt = np.concatenate((train_data, data_augment), axis=3)

            # train
            w, mean_temp = TRCA_train(trainData_pt)
            #
            train_w['bank' + str(idx_filter)] = w
            train_meantemp['bank' + str(idx_filter)] = mean_temp

        # test:
        predictAll = np.zeros((test.shape[0], nEvent))
        flag = 0
        for isplit in test:
            rrall = np.zeros((nEvent, nEvent))
            for idx_filter in range(num_filter):
                idx_filter += 1
                bank_data = filtered_data['bank' + str(idx_filter)]
                test_data = bank_data[:, :, :, isplit]
                test_data = test_data[:, task_point, :]
                rr = TRCA_test(test_data, train_w['bank' + str(idx_filter)],
                               train_meantemp['bank' + str(idx_filter)], True)
                rrall += np.multiply(np.sign(rr), (rr ** 2)) * (idx_filter ** (-1.25) + 0.25)
            predict = np.argmax(rrall, -1)
            predictAll[flag, :] = predict
            flag += 1
        acc_s = acc_calculate(predictAll) + acc_s
    acc = acc_s / n_splits
    # print('sub', idx_num + 1, ', acc = ', acc_s / n_splits)

    return acc

## Result

### eTRCA(w/oSAME)

In [7]:
Aug_size = [0,0,0]
acc_all = np.zeros((70,3))
for i_train in range(3):
    print('train_size=',i_train+1)
    c = time.time()
    acc = Parallel(n_jobs=-1)(delayed(beta_eTRCA_Aug)(idx_num, n_train=i_train+1, t_task=0.5, n_Aug=Aug_size[i_train]) for idx_num in range(70))
    acc =  np.array(acc)
    acc_all[:,i_train] = acc
    e = time.time()
    print('time =', e - c,' s')
    print(acc)
    print('mean_acc:',np.mean(acc))
sio.savemat(r'demo_beta_eTRCA_withoutSAME.mat', {'acc': acc_all})

train_size= 1
time = 106.3842568397522  s
[0.11458333 0.05208333 0.34166667 0.10833333 0.09791667 0.06041667
 0.16666667 0.07916667 0.21458333 0.12916667 0.03333333 0.2375
 0.36666667 0.06666667 0.03125    0.05416667 0.01666667 0.18125
 0.08958333 0.05833333 0.13541667 0.06041667 0.31458333 0.10208333
 0.17916667 0.06875    0.15       0.25416667 0.07708333 0.10416667
 0.1625     0.04375    0.05625    0.09375    0.15416667 0.20416667
 0.07916667 0.03333333 0.08333333 0.0375     0.07083333 0.23333333
 0.05       0.04791667 0.04166667 0.06041667 0.03333333 0.13125
 0.1125     0.05       0.10625    0.07291667 0.325      0.05625
 0.03333333 0.40625    0.20416667 0.24166667 0.01041667 0.225
 0.02708333 0.11875    0.15833333 0.11875    0.08333333 0.13958333
 0.1        0.07916667 0.03541667 0.06666667]
mean_acc: 0.11904761904761903
train_size= 2
time = 95.07941007614136  s
[0.85     0.88125  0.68125  0.6125   0.665625 0.615625 0.621875 0.415625
 0.865625 0.403125 0.125    0.684375 0.728125 0.

### eTRCA(W/SAME)

In [8]:
Aug_size = [3,5,6]
acc_all = np.zeros((70,3))
for i_train in range(3):
    print('train_size=',i_train+1)
    c = time.time()
    acc = Parallel(n_jobs=-1)(delayed(beta_eTRCA_Aug)(idx_num, n_train=i_train+1, t_task=0.5, n_Aug=Aug_size[i_train]) for idx_num in range(70))
    acc =  np.array(acc)
    acc_all[:,i_train] = acc
    e = time.time()
    print('time =', e - c,' s')
    print(acc)
    print('mean_acc:',np.mean(acc))
sio.savemat(r'demo_beta_eTRCA_withSAME.mat', {'acc': acc_all})

train_size= 1
time = 99.01571726799011  s
[0.83958333 0.83333333 0.67916667 0.5625     0.65416667 0.56041667
 0.51041667 0.35625    0.86666667 0.41041667 0.11041667 0.66875
 0.73958333 0.50833333 0.4875     0.53958333 0.14583333 0.83541667
 0.64166667 0.32083333 0.62708333 0.72291667 0.81458333 0.5125
 0.5625     0.21041667 0.61041667 0.56041667 0.45       0.725
 0.3375     0.23125    0.2625     0.68125    0.63541667 0.7375
 0.69791667 0.10208333 0.4625     0.50416667 0.1875     0.59375
 0.3625     0.2        0.46666667 0.36875    0.1625     0.68541667
 0.72708333 0.37083333 0.41041667 0.63541667 0.52291667 0.45208333
 0.13125    0.4875     0.76875    0.59166667 0.17291667 0.6
 0.15       0.65       0.84375    0.51041667 0.16875    0.6875
 0.81458333 0.7375     0.66041667 0.72083333]
mean_acc: 0.5222916666666666
train_size= 2
time = 97.92097282409668  s
[0.909375 0.896875 0.746875 0.728125 0.83125  0.6875   0.70625  0.525
 0.921875 0.509375 0.203125 0.74375  0.821875 0.66875  0.58125  

### TDCA(w/oSAME)

In [9]:
Aug_size = [0,0,0]
acc_all = np.zeros((70,3))
for i_train in range(3):
    print('train_size=',i_train+1)
    c = time.time()
    acc = Parallel(n_jobs=-1)(delayed(beta_TDCA_Aug)(idx_num, n_train=i_train+1, t_task=0.5, n_Aug=Aug_size[i_train]) for idx_num in range(70))
    acc =  np.array(acc)
    acc_all[:,i_train] = acc
    e = time.time()
    print('time =', e - c,' s')
    print(acc)
    print('mean_acc:',np.mean(acc))
sio.savemat(r'demo_beta_TDCA_withoutSAME.mat', {'acc': acc_all})

train_size= 1
time = 303.67482328414917  s
[0.32291667 0.17083333 0.33958333 0.11666667 0.14791667 0.10416667
 0.1375     0.14375    0.43333333 0.2        0.09166667 0.4125
 0.44583333 0.08958333 0.08125    0.10833333 0.02083333 0.43125
 0.23125    0.07916667 0.22291667 0.17083333 0.44583333 0.21666667
 0.20416667 0.08541667 0.27083333 0.25833333 0.11458333 0.2375
 0.2        0.01666667 0.09583333 0.22708333 0.22291667 0.32083333
 0.225      0.0375     0.1125     0.09375    0.0875     0.30833333
 0.07916667 0.09791667 0.0875     0.07916667 0.06666667 0.32291667
 0.2875     0.06041667 0.19375    0.13958333 0.25625    0.07083333
 0.05833333 0.39166667 0.27708333 0.25       0.07916667 0.29375
 0.0375     0.15416667 0.26666667 0.18333333 0.07916667 0.18958333
 0.19583333 0.2125     0.05833333 0.1625    ]
mean_acc: 0.1844940476190476
train_size= 2
time = 222.89798712730408  s
[0.925    0.915625 0.740625 0.721875 0.81875  0.634375 0.71875  0.540625
 0.9375   0.46875  0.1125   0.690625 0.7937

### TDCA(w/SAME)

In [10]:
Aug_size = [3,5,6]
acc_all = np.zeros((70,3))
for i_train in range(3):
    print('train_size=',i_train+1)
    c = time.time()
    acc = Parallel(n_jobs=-1)(delayed(beta_TDCA_Aug)(idx_num, n_train=i_train+1, t_task=0.5, n_Aug=Aug_size[i_train]) for idx_num in range(70))
    acc =  np.array(acc)
    acc_all[:,i_train] = acc
    e = time.time()
    print('time =', e - c,' s')
    print(acc)
    print('mean_acc:',np.mean(acc))
sio.savemat(r'demo_beta_TDCA_withSAME.mat', {'acc': acc_all})

train_size= 1
time = 303.69347047805786  s
[0.80625    0.83125    0.68958333 0.61041667 0.75833333 0.52291667
 0.66041667 0.36875    0.88958333 0.41041667 0.11875    0.66666667
 0.7375     0.49375    0.47708333 0.46875    0.20833333 0.81666667
 0.6875     0.37291667 0.64583333 0.64583333 0.82916667 0.55833333
 0.53333333 0.26041667 0.59583333 0.56666667 0.46875    0.79791667
 0.33125    0.23958333 0.29166667 0.70416667 0.64375    0.75416667
 0.70416667 0.13541667 0.48958333 0.46041667 0.18541667 0.55
 0.3375     0.19375    0.48958333 0.33958333 0.16458333 0.66458333
 0.70416667 0.43958333 0.43333333 0.6125     0.62291667 0.42916667
 0.18958333 0.67291667 0.70833333 0.6125     0.14583333 0.58958333
 0.18333333 0.575      0.80833333 0.55833333 0.15625    0.69166667
 0.81666667 0.68541667 0.54583333 0.72291667]
mean_acc: 0.5297321428571431
train_size= 2
time = 235.44340467453003  s
[0.93125  0.921875 0.74375  0.73125  0.85     0.6625   0.703125 0.5125
 0.91875  0.540625 0.16875  0.725    